## Set-up workspace

In [ ]:
!pip install geopandas
!pip install geemap

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
import geemap
# geemap.update_package()
import ee

In [ ]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


## Load in study area

In [ ]:
knp = ee.FeatureCollection('users/jdmwhite/KNP_study_area')

## Load in variables from Joe

### Set-up date variables

In [ ]:
startDate = ee.Date('2013-01-01')
endDate = ee.Date('2013-12-01')

### Vegetation indices

In [ ]:
# MODIS EVI
summerStart = ee.Date('2012-11-01')
summerEnd = ee.Date('2013-04-01')
winterStart = ee.Date('2013-05-01')
winterEnd = ee.Date('2013-10-01')

modis_evi_summer = ee.ImageCollection("MODIS/006/MOD13Q1").filterDate(summerStart, summerEnd).select('EVI').mean().multiply(0.0001).clip(knp).rename('evi_summer')
modis_evi_winter = ee.ImageCollection("MODIS/006/MOD13Q1").filterDate(winterStart, winterEnd).select('EVI').mean().multiply(0.0001).clip(knp).rename('evi_winter')

### Temperature

In [ ]:
# Temperature from Bioclim
bio_minT = ee.Image("WORLDCLIM/V1/BIO").select(["bio06"]).multiply(0.1).rename('bio_minT').clip(knp)
bio_maxT = ee.Image("WORLDCLIM/V1/BIO").select(["bio05"]).multiply(0.1).rename('bio_maxT').clip(knp)

### Rainfall

In [ ]:
# Rainfall from Bioclim
bio_annP = ee.Image("WORLDCLIM/V1/BIO").select(["bio12"]).rename('bio_annP').clip(knp)
bio_Pwq = ee.Image("WORLDCLIM/V1/BIO").select(["bio16"]).rename('bio_Pwq').clip(knp)
bio_Pdq = ee.Image("WORLDCLIM/V1/BIO").select(["bio17"]).rename('bio_Pdq').clip(knp)

### Topography

In [ ]:
# elevation
elevation = ee.Image("NASA/NASADEM_HGT/001").select('elevation').clip(knp)

# geomorpho90m
cti = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/cti").median().clip(knp).rename('cti')
tri = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/tri").median().clip(knp).rename('tri')
slope = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/slope").median().clip(knp).rename('slope')
vrm = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/vrm").median().clip(knp).rename('vrm')
roughness = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/roughness").median().clip(knp).rename('roughness')
tpi = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/tpi").median().clip(knp).rename('tpi')
spi = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/spi").median().clip(knp).rename('spi')

# microclimate CHILI
chili = ee.Image('CSP/ERGo/1_0/Global/SRTM_CHILI').rename('CHILI').clip(knp)

# topographic diversity
topD = ee.Image('CSP/ERGo/1_0/Global/SRTM_topoDiversity').rename('topD').clip(knp)

### Soils

In [ ]:
# HiHydro Layers 250m
ksat = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/ksat").first().multiply(0.0001).rename('ksat').clip(knp)
satfield = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/sat-field").first().multiply(0.0001).rename('satfield').clip(knp)
N = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/N").first().multiply(0.0001).rename('N').clip(knp)
alpha = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/alpha").first().multiply(0.0001).rename('alpha').clip(knp)
crit_wilt = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/crit-wilt").first().multiply(0.0001).rename('critwilt').clip(knp)
field_cirt = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/field-crit").first().multiply(0.0001).rename('fieldcrit').clip(knp)
ormc = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/ormc").first().multiply(0.0001).rename('ormc').clip(knp)
stc = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/stc").first().multiply(0.0001).rename('stc').clip(knp)
wcavail = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/wcavail").first().multiply(0.0001).rename('wcavail').clip(knp)
wcpf2 = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/wcpf2").first().multiply(0.0001).rename('wcpf2').clip(knp)
wcpf3 = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/wcpf3").first().multiply(0.0001).rename('wcpf3').clip(knp)
wcpf4_2 = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/wcpf4-2").first().multiply(0.0001).rename('wcpf4-2').clip(knp)
# wcres = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/wcres").first().multiply(0.0001).rename('wcres').clip(knp)
wcsat = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/wcsat").first().multiply(0.0001).rename('wcsat').clip(knp)

# Hengl Africa Soils 30m
bedrock_depth = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/bedrock_depth").select('b1').rename('H_bed_depth').clip(knp)
bulk_density = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/bulk_density").select('b1').rename('H_bulk_dens').clip(knp)
carbon_organic = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/carbon_organic").select('b1').rename('H_carb_org').clip(knp)
carbon_total = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/carbon_total").select('b1').rename('H_carb_tot').clip(knp)
cation_exchange_capacity = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/cation_exchange_capacity").select('b1').rename('H_cec').clip(knp)
clay_content = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/clay_content").select('b1').rename('H_clay').clip(knp)
fertility_capability_classification = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/fcc").select('b1').rename('H_fcc').clip(knp)
nitrogen_total = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/nitrogen_total").select('b1').rename('H_N_tot').clip(knp)
ph = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/ph").select('b1').rename('H_ph').clip(knp)
sand_content = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/sand_content").select('b1').rename('H_sand').clip(knp)
silt_content = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/silt_content").select('b1').rename('H_silt').clip(knp)
stone_content = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/stone_content").select('b1').rename('H_stone').clip(knp)
texture_class = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/texture_class").select('b1').rename('H_text_class').clip(knp)

## Load in Variables from Geethen

### Water

In [ ]:
# GSW
gsw = ee.Image("JRC/GSW1_3/GlobalSurfaceWater").clip(knp)
# dropping max_extent
gsw = gsw.select(['occurrence', 'change_abs', 'change_norm', 'seasonality', 'recurrence', 'transition'])

# Pickens
gsd = ee.Image('projects/glad/water/dynamics20/classes').clip(knp)

# OSM
osm_water = ee.ImageCollection("projects/sat-io/open-datasets/OSM_waterLayer").median().clip(knp).rename('osm_water')

### Infrastructure

In [ ]:
# GRIP
grip4_africa = ee.FeatureCollection("projects/sat-io/open-datasets/GRIP4/Africa").filterBounds(knp).distance().clip(knp).rename('distance_roads')

# Transmission
transmission = ee.FeatureCollection("projects/sat-io/open-datasets/predictive-global-power-system/distribution-transmission-lines")\
                .filterBounds(knp).distance().clip(knp).rename('distance_trans_lines')

# Settlements
S2_CNN_collection = ee.ImageCollection('users/ghsl/S2_CNN');
mosaic = S2_CNN_collection.filterBounds(knp).mosaic().clip(knp);

def layerMasked (layer):
  return layer.updateMask(layer.gte(20))

settlements = layerMasked(mosaic).gt(0).selfMask().reduceToVectors(**{'geometry':knp,'scale':10, 'maxPixels':1e13}).distance()\
                .rename('distance_settlements')

### Soil erosion

In [ ]:
# bare ground
bare_frequency = ee.Image('users/geocis/BareSurfaces/BF_1980_2019').clip(knp)

### Landcover

In [ ]:
# SA LC 2018
sa_nlc2018 = ee.Image('projects/sat-io/open-datasets/landcover/SA_NLC_2018').clip(knp).rename('SANLC')
# dict = {"names": ["Contiguous (indigenous) Forest (combined very high, high, medium)", "Contiguous Low Forest & Thicket (combined classes)", "Dense Forest & Woodland (35 - 75% cc)", "Open Woodland (10 - 35% cc)", "Contiguous & Dense Planted Forest (combined classes)", "Open & Sparse Planted Forest", "Temporary Unplanted Forest", "Low Shrubland (other regions)", "Low Shrubland (Fynbos)", "Low Shrubland (Succulent Karoo)", "Low Shrubland (Nama Karoo)", "Sparsely Wooded Grassland (5 - 10% cc)", "Natural Grassland", "Natural Rivers", "Natural Estuaries & Lagoons", "Natural Ocean, Coastal", "Natural Lakes", "Natural Pans (flooded @ obsv time)", "Artificial Dams (incl. canals)", "Artificial Sewage Ponds", "Artificial Flooded Mine Pits", "Herbaceous Wetlands (currently mapped)", "Herbaceous Wetlands (previous mapped extent)", "Mangrove Wetlands", "Natural Rock Surfaces", "Dry Pans", "Eroded Lands", "Sand Dunes (terrestrial)", "Coastal Sand Dunes & Beach Sand", "Bare Riverbed Material", "Other Bare", "Cultivated Commercial Permanent Orchards", "Cultivated Commercial Permanent Vines", "Cultivated Commercial Sugarcane Pivot Irrigated", "Commercial Permanent Pineapples", "Cultivated Commercial Sugarcane Non-Pivot (all other)", "Cultivated Emerging Farmer Sugarcane Non-Pivot (all other)", "Commercial Annuals Pivot Irrigated", "Commercial Annuals Non-Pivot Irrigated", "Commercial Annuals Crops Rain-Fed / Dryland / Non-Irrigated", "Subsistence / Small-Scale Annual Crops", "Fallow Land & Old Fields (Trees)", "Fallow Land & Old Fields (Bush)", "Fallow Land & Old Fields (Grass)", "Fallow Land & Old Fields (Bare)", "Fallow Land & Old Fields (Low Shrub)", "Residential Formal (Tree)", "Residential Formal (Bush)", "Residential Formal (low veg / grass)", "Residential Formal (Bare)", "Residential Informal (Tree)", "Residential Informal (Bush)", "Residential Informal (low veg / grass)", "Residential Informal (Bare)", "Village Scattered (bare only)", "Village Dense (bare only)", "Smallholdings (Tree)", "Smallholdings (Bush)", "Smallholdings (low veg / grass)", "Smallholdings (Bare)", "Urban Recreational Fields (Tree)", "Urban Recreational Fields (Bush)", "Urban Recreational Fields (Grass)", "Urban Recreational Fields (Bare)", "Commercial", "Industrial", "Roads & Rail (Major Linear)", "Mines: Surface Infrastructure", "Mines: Extraction Sites: Open Cast & Quarries combined", "Mines: Extraction Sites: Salt Mines", "Mines: Waste (Tailings) & Resource Dumps", "Land-fills", "Fallow Land & Old Fields (wetlands)"], "colors": ["#F2F2F2", "#065106", "#005F00", "#008500", "#F74006", "#F9764D", "#F9906C", "#B8ABD1", "#8FAB39", "#AC92C5", "#AC9CDA", "#85D285", "#D2B485", "#00009F", "#041FA7", "#0639AB", "#0D50AC", "#125FAC", "#1373B4", "#1D81B6", "#1F8EB8", "#06DEDC", "#06E0D0", "#9F1FEC", "#ffffe0", "#DCDAC5", "#F9E0E0", "#F9F9C5", "#F9F9A7", "#CDD2E0", "#ffffe0", "#A62C39", "#B31F5C", "#DB0000", "#9F3978", "#FF0000", "#F64D6C", "#381A12", "#521F1C", "#85402C", "#C5735F", "#C1436C", "#C55E82", "#D27592", "#E0AAB8", "#DB90A9", "#ECDB0F", "#F6EC13", "#F9F81F", "#FFFF29", "#EC82EC", "#F691E0", "#F99FCF", "#FFC5CF", "#ECC500", "#FFD91F", "#AC7879", "#B89192", "#C49C9E", "#D2B8B8", "#BFFF00", "#33FF33", "#66FF66", "#99FF99", "#C49F0D", "#8F8506", "#F9DD03", "#FFFF00", "#B30606", "#C50606", "#D21D1A", "#F95479", "#6CE7DC"]}

# Corine LC
LC = ee.Image(ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V-C3/Global").first().clip(knp)).select('.*coverfraction')\
.select('bare-coverfraction', 'urban-coverfraction', 'crops-coverfraction', 'grass-coverfraction', 'water-permanent-coverfraction', 'water-seasonal-coverfraction', 'shrub-coverfraction', 'tree-coverfraction')

# SA Vegmap 2019b
vegmap =  ee.Image(ee.FeatureCollection("users/zandersamuel/SA_misc/SA_veg_functional_V3_26032019").reduceToImage(**{\
    'properties': ['BRGNID_18'],\
    'reducer': ee.Reducer.first()\
})).rename('vegtype_id')

### Habitat heterogeneity

In [ ]:
# Global habitat heterogeneity
cov = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/coefficient_of_variation_1km")
contrast = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/contrast_1km")
corr = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/correlation_1km")
dissimilarity = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/dissimilarity_1km")
entropy = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/entropy_1km")
homogeneity = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/homogeneity_1km")
maximum = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/maximum_1km")
mean = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/mean_1km")
pielou = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/pielou_1km")
range = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/range_1km")
shannon = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/shannon_1km")
simpson = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/simpson_1km")
sd = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/standard_deviation_1km")
uniformity = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/uniformity_1km")
variance = ee.Image("projects/sat-io/open-datasets/global_habitat_heterogeneity/variance_1km")

hab_hetero = ee.Image.cat([cov, contrast, corr, dissimilarity, entropy, homogeneity, maximum, mean, pielou, range, shannon, simpson, sd, uniformity, variance])\
                .rename(['cov', 'contrast', 'corr', 'dissimilarity', 'entropy', 'homogeneity', 'maximum', 'mean', 'pielou', 'range', 'shannon', 'simpson', 'sd', 'uniformity', 'variance'])\
                .clip(knp)

# print(hab_hetero.bandNames().getInfo())

# Global canopy height
canopy_height = ee.ImageCollection('users/potapovpeter/GEDI_V27').filterBounds(knp).mosaic().clip(knp).rename('canopy_height')


### Fire

In [ ]:
FireCCI = ee.ImageCollection('ESA/CCI/FireCCI/5_1').select(['BurnDate','ConfidenceLevel'])


# Set your start and end dates to filter by
startDate = ee.Date.fromYMD(2001,1,1)
endDate = ee.Date.fromYMD(2018,12,31)
years = ee.List.sequence(2001, 2018)

# Select appropriate bands, filter by dates, clip by AOI, set year and yrmnth properties
fire = FireCCI.filterDate(startDate, endDate)\
    .map(lambda img : img.clip(knp)\
              .set('year', ee.Image(img).date().get('year'))\
              .set('yrmnth',ee.Date.parse('YYYY_MM_DD', (img.get('system:index'))).format('YYYY_MM')))

# Remove all fires with less than 50% confidence
confMask = fire.map(lambda img: img.select('BurnDate').updateMask(img.select('ConfidenceLevel').gt(50)\
                    .set('year', ee.Image(img).date().get('year')))\
                    .set('yrmnth', ee.Image(img).get('yrmnth')))

# Count the distinct DOY the occurrences of those fires AND Set the band property to year
fireCNT = ee.ImageCollection.fromImages(years.map(lambda year : confMask.filterMetadata('year', 'equals', year)\
    .select('BurnDate').reduce(ee.Reducer.countDistinct()).set('year', year))).sum().unmask()

# Concatenate the DOY per month into a new image collection
fireDOY = ee.ImageCollection.fromImages(years.map(lambda year: confMask.filterMetadata('year', 'equals', year)\
    .select('BurnDate').reduce(ee.Reducer.firstNonNull()).set('year', year))).mode() 

latest_fire = confMask.qualityMosaic('BurnDate').unmask()

## Merge covariates as stack/multiband image

In [ ]:
# covariates = minT.addBands([wSeasonality, totalPrecipitation ,D2Coast,elevation,chili,GHM,topD, frost, FDs, FDs10])

covariates = elevation.addBands([modis_evi_summer, modis_evi_winter, bio_minT, bio_maxT, bio_annP, bio_Pwq,\
bio_Pdq, cti, tri, slope, vrm, roughness, tpi, spi, chili, topD, ksat, satfield, N, alpha, crit_wilt,\
field_cirt, ormc, stc, wcavail, wcpf2, wcpf3, wcpf4_2, wcsat, bedrock_depth, bulk_density,\
carbon_organic, carbon_total, cation_exchange_capacity, clay_content, fertility_capability_classification,\
nitrogen_total, ph, sand_content, silt_content, stone_content, texture_class,\
\
gsw, gsd, osm_water, grip4_africa, transmission, settlements, bare_frequency, sa_nlc2018, LC, vegmap,\
# cov, contrast, corr, dissimilarity, entropy, homogeneity, maximum, mean, pielou, range, shannon, simpson,\
# sd, uniformity, variance,\
hab_hetero, canopy_height, fireCNT, latest_fire])

print(covariates.bandNames().getInfo())

['elevation', 'evi_summer', 'evi_winter', 'bio_minT', 'bio_maxT', 'bio_annP', 'bio_Pwq', 'bio_Pdq', 'cti', 'tri', 'slope', 'vrm', 'roughness', 'tpi', 'spi', 'CHILI', 'topD', 'ksat', 'satfield', 'N', 'alpha', 'critwilt', 'fieldcrit', 'ormc', 'stc', 'wcavail', 'wcpf2', 'wcpf3', 'wcpf4-2', 'wcsat', 'H_bed_depth', 'H_bulk_dens', 'H_carb_org', 'H_carb_tot', 'H_cec', 'H_clay', 'H_fcc', 'H_N_tot', 'H_ph', 'H_sand', 'H_silt', 'H_stone', 'H_text_class', 'occurrence', 'change_abs', 'change_norm', 'seasonality', 'recurrence', 'transition', 'wp', 'osm_water', 'distance_roads', 'distance_trans_lines', 'distance_settlements', 'SF', 'SANLC', 'bare-coverfraction', 'urban-coverfraction', 'crops-coverfraction', 'grass-coverfraction', 'water-permanent-coverfraction', 'water-seasonal-coverfraction', 'shrub-coverfraction', 'tree-coverfraction', 'vegtype_id', 'cov', 'contrast', 'corr', 'dissimilarity', 'entropy', 'homogeneity', 'maximum', 'mean', 'pielou', 'range', 'shannon', 'simpson', 'sd', 'uniformity', 

## Export covariates to GEE asset

In [ ]:
task = ee.batch.Export.image.toAsset(**{'image':covariates,'description':'covs_1000', \
'assetId':'users/jdmwhite/WTWTA_covariates_1000','region':knp.geometry(), 'scale': 1000,\
 'maxPixels':1e13})
task.start()

In [ ]:
task.status()

{'state': 'COMPLETED',
 'description': 'covs_1000',
 'creation_timestamp_ms': 1625689807776,
 'update_timestamp_ms': 1625689892536,
 'start_timestamp_ms': 1625689858719,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://code.earthengine.google.com/?asset=projects/earthengine-legacy/assets/users/jdmwhite/WTWTA_covariates_1000'],
 'attempt': 1,
 'id': 'PZBD3P52U5B3QLS4262GWTTF',
 'name': 'projects/earthengine-legacy/operations/PZBD3P52U5B3QLS4262GWTTF'}

### Read extracted covariates

In [ ]:
covariates = ee.Image('users/jdmwhite/WTWTA_covariates')
print(covariates.bandNames().getInfo())

['elevation', 'evi_summer', 'evi_winter', 'bio_minT', 'bio_maxT', 'bio_annP', 'bio_Pwq', 'bio_Pdq', 'cti', 'tri', 'slope', 'vrm', 'roughness', 'tpi', 'spi', 'CHILI', 'topD', 'ksat', 'satfield', 'N', 'alpha', 'critwilt', 'fieldcrit', 'ormc', 'stc', 'wcavail', 'wcpf2', 'wcpf3', 'wcpf4-2', 'wcres', 'wcsat', 'H_bed_depth', 'H_bulk_dens', 'H_carb_org', 'H_carb_tot', 'H_cec', 'H_clay', 'H_fcc', 'H_N_tot', 'H_ph', 'H_sand', 'H_silt', 'H_stone', 'H_text_class', 'occurrence', 'change_abs', 'change_norm', 'seasonality', 'recurrence', 'transition', 'max_extent', 'wp', 'osm_water', 'distance_roads', 'distance_trans_lines', 'distance_settlements', 'SF', 'SANLC', 'bare-coverfraction', 'urban-coverfraction', 'crops-coverfraction', 'grass-coverfraction', 'moss-coverfraction', 'water-permanent-coverfraction', 'water-seasonal-coverfraction', 'shrub-coverfraction', 'snow-coverfraction', 'tree-coverfraction', 'vegtype_id', 'cov', 'contrast', 'corr', 'dissimilarity', 'entropy', 'homogeneity', 'maximum', 'm

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=232f2193-6834-457f-a7ff-77b7f2e92f86' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>